# Topik: Analisis Sentimen Opini Publik terhadap Kebijakan Pemindahan Dana Rp 200 Triliun ke Bank Himbara oleh Menteri Keuangan Purbaya melalui Komentar YouTube

## Notebook: Preprocessing Data dari data hasil filtering.

**Anggota Kelompok:**  
- 140810230011 - Lukas Austin  
- 140810230045 - Devin Suryadi  
- 140810230057 - Orlando Bloem Sutono  

Mata Kuliah: Data Mining  
Kelas: A  
Dosen: Bu Helen

---

## 1. Import Library
Library yang dibutuhkan untuk preprocessing teks.

In [1]:
import os
import re
import time
import pandas as pd
import unicodedata
from bs4 import BeautifulSoup
import emoji
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from indoNLP.preprocessing import replace_slang
# from indoNLP.preprocessing import remove_stopwords

## 2. Load Dataset
Memuat data hasil filtering komentar dari YouTube.

In [2]:
pd.set_option('display.max_colwidth', 200)

df = pd.read_csv("dataset_filtered.csv")
print(f"Dataset dimuat: {len(df)} baris, kolom: {list(df.columns)}")

df = df[['comment']].copy()
df.head()

Dataset dimuat: 924 baris, kolom: ['id_comment', 'comment', 'author', 'like_count', 'time', 'video_id']


,comment
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak."
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu."
2,"nyatanya bank2 himbara masih selektif mengucurkan dana untuk UMKM yg sangat membutuhkan tambahan modal, seperti yg saya alami. punya usaha percetakan sdh berjalan sejak 2002 sampai sekarang membut..."
3,Ayooo pak pur ttp semangat bangun indonesia 👍
4,Ayo rakyat cerdas harus mengonyrol pemaik penggunaan uang kenapa negara kaya rakyatnya mismin pemainya banysk rekayasa yg serong ngeri ngeri rskyat jadi korban


## 3. Text Cleaning
Melakukan pembersihan HTML, URL, emoji, tanda baca, dan karakter berulang.

In [3]:
# Menghapus HTML dan tag tidak relevan
def remove_html(text):
    return BeautifulSoup(str(text), "html.parser").get_text()

# Menormalisasi unicode ke bentuk konsisten
def normalize_unicode(text):
    return unicodedata.normalize('NFKC', str(text))

# Menghapus URL, mention, dan hashtag
def remove_url_mentions(text):
    return re.sub(r"http\S+|www\S+|@\w+|#\w+", " ", str(text))

# Menghapus emoji dari teks
def remove_emoji(text):
    return emoji.replace_emoji(str(text), replace=' ')

# Mengubah teks ke lowercase dan menghapus tanda baca
def lowercase_and_remove_punct(text):
    return re.sub(r'[^a-z0-9\s]', ' ', str(text).lower())

# Menormalkan spasi berlebih dan huruf yang berulang
def normalize_space_and_repetition(text):
    text = re.sub(r'(.)\1{2,}', r'\1\1', str(text))
    return re.sub(r'\s+', ' ', text).strip()

In [4]:
df['clean_text'] = df['comment'].apply(remove_html)
df['clean_text'] = df['clean_text'].apply(normalize_unicode)
df['clean_text'] = df['clean_text'].apply(remove_url_mentions)
df['clean_text'] = df['clean_text'].apply(remove_emoji)
df['clean_text'] = df['clean_text'].apply(lowercase_and_remove_punct)
df['clean_text'] = df['clean_text'].apply(normalize_space_and_repetition)

df[['comment', 'clean_text']].head()

,comment,clean_text
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",seperti langit dan bumi kerja pak purbaya dan bu sri mulyani senang lihat kerja bpk sehat semangat ya pak
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",semoga purbaya selamat ngak dikerjain sama oligarki para mafia di sektor energy pangan dan keuangan yg merasa kepentingannya terganggu
2,"nyatanya bank2 himbara masih selektif mengucurkan dana untuk UMKM yg sangat membutuhkan tambahan modal, seperti yg saya alami. punya usaha percetakan sdh berjalan sejak 2002 sampai sekarang membut...",nyatanya bank2 himbara masih selektif mengucurkan dana untuk umkm yg sangat membutuhkan tambahan modal seperti yg saya alami punya usaha percetakan sdh berjalan sejak 2002 sampai sekarang membutuh...
3,Ayooo pak pur ttp semangat bangun indonesia 👍,ayoo pak pur ttp semangat bangun indonesia
4,Ayo rakyat cerdas harus mengonyrol pemaik penggunaan uang kenapa negara kaya rakyatnya mismin pemainya banysk rekayasa yg serong ngeri ngeri rskyat jadi korban,ayo rakyat cerdas harus mengonyrol pemaik penggunaan uang kenapa negara kaya rakyatnya mismin pemainya banysk rekayasa yg serong ngeri ngeri rskyat jadi korban


## 4. Normalization
Mengganti kata tidak baku (gaul), typo, dan kata dengan makna yang sama namun berbeda penulisan menjadi kata yang sama dan benar.

##### 4.1 Daftar kata typo yang ada di dataset melalui eksplorasi dataset secara manual.

In [5]:
typo_dict = {
    "menkeu": "menteri keuangan", "mentri": "menteri", "resiko": "risiko",
    "duit": "uang", "bi": "bank indonesia", "kur": "kredit usaha rakyat",
    "pinjol": "pinjaman online", "bumn": "badan usaha milik negara",
    "umkm": "usaha mikro kecil menengah", "dpr": "dewan perwakilan rakyat",
    "pmerintah": "pemerintah", "pemerintahn": "pemerintahan", "krdit": "kredit",
    "bri": "bank rakyat indonesia", "himbara": "himpunan bank milik negara",
    "menku": "menteri keuangan", "pur": "purbaya", "rskyat": "rakyat",
    
    "ngak": "tidak", "padahal": "walaupun begitu", "usah": "usaha", "percetaka": "percetakan",
    "bunganya": "bunga", "mengonyrol": "mengontrol", "pemaik": "pemain", "mismin": "miskin",
    
    "pak": "bapak", "bpk.": "bapak", "bu": "ibu", "bunda": "ibu", "bpk": "bapak",
    "bapaknya": "bapak", "ibunya": "ibu", "mas": "saudara", "mba": "saudari",
    "mbak": "saudari", "om": "saudara", "bang": "saudara", "ekonomj": "ekonomi",
    
    "g": "tidak", "gaes": "teman teman", "selamet": "selamat", "plis": "tolong",
    "sehattt": "sehat", "terimakasih": "terima kasih", "terimakasi": "terima kasih",
    "tqsm": "terima kasih banyak", "luarbiasa": "luar biasa", "banysk": "banyak", "enggak": "tidak",
    
    "alah": "allah", "alhamdulilah": "alhamdulillah", "bukanya": "bukannya",
    "mengerakkan": "menggerakkan", "menggerakan": "menggerakkan", "ngangur": "nganggur",
    "perusahan": "perusahaan", "energy": "energi", "ksn": "kesana", "kemana2": "kemana mana",

    "yag": "yang", "rakyatt": "rakyat", "trakyat": "rakyat", "keredit": "kredit",
    "hnya": "hanya", "mbuat": "membuat", "dri": "dari", "bungax": "bunga", "jdi": "jadi",
    "lagih": "lagi", "smudah": "semudah", "kerjaa": "kerja", "slama": "selama", "sya": "saya",
    "dbagi": "dibagi", "khan": "kan", "ajah": "aja", "tnp": "tanpa", "tpi": "tapi", "diaa": "dia",
    "modall": "modal", "kslo": "kalau", "amanahh": "amanah","perbankkan": "perbankan", "pemainya": "pemain"
}

##### 4.2 Fungsi Normalisasi

In [6]:
def normalize(text):
    text = replace_slang(text)  # ubah kata slang ke bentuk baku dengan indoNLP
    words = text.split()
    normalized = [typo_dict.get(w, w) for w in words]  # ganti typo sesuai dictionary typo_dict
    return " ".join(normalized)


In [7]:
df['clean_text'] = df['clean_text'].apply(normalize)  # normalisasi typo dan slang
df[['comment', 'clean_text']].head()

,comment,clean_text
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",seperti langit dan bumi kerja bapak purbaya dan ibu sri mulyani senang lihat kerja bapak sehat semangat ya bapak
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",semoga purbaya selamat tidak dikerjain sama oligarki para mafia di sektor energi pangan dan keuangan yang merasa kepentingannya terganggu
2,"nyatanya bank2 himbara masih selektif mengucurkan dana untuk UMKM yg sangat membutuhkan tambahan modal, seperti yg saya alami. punya usaha percetakan sdh berjalan sejak 2002 sampai sekarang membut...",nyatanya bank2 himpunan bank milik negara masih selektif mengucurkan dana untuk usaha mikro kecil menengah yang sangat membutuhkan tambahan modal seperti yang saya alami punya usaha percetakan sud...
3,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo bapak purbaya tetap semangat bangun indonesia
4,Ayo rakyat cerdas harus mengonyrol pemaik penggunaan uang kenapa negara kaya rakyatnya mismin pemainya banysk rekayasa yg serong ngeri ngeri rskyat jadi korban,ayo rakyat cerdas harus mengontrol pemain penggunaan uang kenapa negara kayak rakyatnya miskin pemain banyak rekayasa yang serong ngeri ngeri rakyat jadi korban


## 5. Stopword Removal dan Stemming
Menghapus kata umum yang tidak informatif dan mengembalikan kata ke bentuk dasarnya.

##### Inisialisasi Stopword dan Stemmer

In [8]:
# Inisialisasi Stopword
stop_factory = StopWordRemoverFactory()
stop_remover = stop_factory.create_stop_word_remover()

# Inisialisasi Stemmer
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

##### Mendefinisikan fungsi untuk melindungi kata negasi sebelum stemming (karena library Sastrawi biasa menghapusnya), serta fungsi pemulihan setelah stemming selesai dilakukan.

In [9]:
# Fungsi proteksi negasi sebelum stemming karena biasanya library ini menghilangkan kata negasi
def protect_negations(text):
    text = re.sub(r"\btidak\b", "NEG-TIDAK", text, flags=re.IGNORECASE)
    text = re.sub(r"\bbukan\b", "NEG-BUKAN", text, flags=re.IGNORECASE)
    text = re.sub(r"\bbelum\b", "NEG-BELUM", text, flags=re.IGNORECASE)
    text = re.sub(r"\bkurang\b", "NEG-KURANG", text, flags=re.IGNORECASE)
    text = re.sub(r"\btanpa\b", "NEG-TANPA", text, flags=re.IGNORECASE)
    return text

# Fungsi pemulihan negasi setelah stemming
def restore_negations(text):
    text = re.sub(r"\bneg-tidak\b", "tidak", text, flags=re.IGNORECASE)
    text = re.sub(r"\bneg-bukan\b", "bukan", text, flags=re.IGNORECASE)
    text = re.sub(r"\bneg-belum\b", "belum", text, flags=re.IGNORECASE)
    text = re.sub(r"\bneg-kurang\b", "kurang", text, flags=re.IGNORECASE)
    text = re.sub(r"\bneg-tanpa\b", "tanpa", text, flags=re.IGNORECASE)
    text = re.sub(r"\s+", " ", text).strip()
    return text

##### Code eksekusi tahap proteksi negasi, penghapusan stopword, stemming, dan pemulihan negasi secara berurutan.

In [10]:
# Proteksi kata negasi
df['clean_text'] = df['clean_text'].apply(protect_negations)

# Stopword removal
df['clean_text'] = df['clean_text'].apply(lambda x: stop_remover.remove(x))

# Stemming
df['clean_text'] = df['clean_text'].apply(lambda x: stemmer.stem(x))

# Pulihkan kata negasi
df['clean_text'] = df['clean_text'].apply(restore_negations)

df[['comment', 'clean_text']].head()

,comment,clean_text
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",langit bumi kerja bapak purbaya ibu sri mulyani senang lihat kerja bapak sehat semangat bapak
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",moga purbaya selamat tidak dikerjain sama oligarki mafia sektor energi pangan uang rasa penting ganggu
2,"nyatanya bank2 himbara masih selektif mengucurkan dana untuk UMKM yg sangat membutuhkan tambahan modal, seperti yg saya alami. punya usaha percetakan sdh berjalan sejak 2002 sampai sekarang membut...",nyata bank2 himpun bank milik negara selektif kucur dana usaha mikro kecil tengah sangat butuh tambah modal alami punya usaha cetak jalan sejak 2002 sekarang butuh modal 25 juta aju kredit usaha r...
3,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo bapak purbaya tetap semangat bangun indonesia
4,Ayo rakyat cerdas harus mengonyrol pemaik penggunaan uang kenapa negara kaya rakyatnya mismin pemainya banysk rekayasa yg serong ngeri ngeri rskyat jadi korban,ayo rakyat cerdas kontrol main guna uang negara kayak rakyat miskin main banyak rekayasa serong ngeri ngeri rakyat jadi korban


## 6. Menambahkan Kolom Sentimen
Menambahkan kolom `sentiment` kosong sebagai placeholder untuk tahap labeling.

In [11]:
# Tambahkan kolom sentiment kosong
df['sentiment'] = ""

df[['comment', 'clean_text', 'sentiment']].head(5)

,comment,clean_text,sentiment
0,"Seperti langit dan bumi, kerja pak Purbaya dan bu Sri mulyani. Senang lihat kerja Bpk. Sehat, semangat ya pak.",langit bumi kerja bapak purbaya ibu sri mulyani senang lihat kerja bapak sehat semangat bapak,
1,"Semoga Purbaya selamat, ngak dikerjain sama oligarki & para mafia di sektor energy, pangan dan keuangan yg merasa kepentingannya terganggu.",moga purbaya selamat tidak dikerjain sama oligarki mafia sektor energi pangan uang rasa penting ganggu,
2,"nyatanya bank2 himbara masih selektif mengucurkan dana untuk UMKM yg sangat membutuhkan tambahan modal, seperti yg saya alami. punya usaha percetakan sdh berjalan sejak 2002 sampai sekarang membut...",nyata bank2 himpun bank milik negara selektif kucur dana usaha mikro kecil tengah sangat butuh tambah modal alami punya usaha cetak jalan sejak 2002 sekarang butuh modal 25 juta aju kredit usaha r...,
3,Ayooo pak pur ttp semangat bangun indonesia 👍,ayo bapak purbaya tetap semangat bangun indonesia,
4,Ayo rakyat cerdas harus mengonyrol pemaik penggunaan uang kenapa negara kaya rakyatnya mismin pemainya banysk rekayasa yg serong ngeri ngeri rskyat jadi korban,ayo rakyat cerdas kontrol main guna uang negara kayak rakyat miskin main banyak rekayasa serong ngeri ngeri rakyat jadi korban,


## 7. Labeling dengan API Gemini
Melakukan labeling otomatis pada seluruh data menggunakan model Gemini 2.0 Flash.
Prompt dibuat agar model memahami konteks kebijakan dan mengklasifikasikan opini publik secara akurat ke dalam tiga kelas sentimen: positif, negatif, dan netral.

##### API client dari Google Gemini.

In [38]:
from google import genai
from dotenv import load_dotenv

load_dotenv(override=True)
GEMINI = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=GEMINI)

In [39]:
def classify_sentiment_batch(comments):
    # Gabungkan komentar dengan separator
    joined_comments = "\n### KOMENTAR ###\n".join(comments)

    # Prompt
    prompt = f"""
    Anda adalah analis sentimen publik. 
    Tugas Anda adalah menentukan apakah setiap komentar berikut memiliki sentimen positif, negatif, atau netral 
    terhadap kebijakan Menteri Keuangan Purbaya yang memindahkan dana Rp 200 triliun ke Bank Himbara.
    Tolong baca dengan cermat setiap komentar dan berikan penilaian yang akurat.
    Jangan ada komentar yang terlewat.

    Kriteria penilaian:
    - Positif berarti mendukung, memuji, atau menilai kebijakan ini baik/bermanfaat.
    - Negatif berarti menentang, mengkritik, atau menilai kebijakan ini buruk/merugikan.
    - Netral berarti informatif, bercanda, atau tidak menunjukkan opini jelas.

    Berikut kumpulan komentar yang perlu diklasifikasikan (dipisahkan oleh teks ### KOMENTAR ###):
    {joined_comments}

    Kembalikan hasil dalam format Python list seperti berikut:
    ["Positif", "Negatif", "Netral", ...]
    dengan urutan yang sesuai komentar di atas.

    Jangan tambahkan teks lain di luar list, INGAT TIDAK ADA SAMA SEKALI TANPA TERKECUALI, Sesuaikan dengan contoh format di atas.
    JANGAN ADA YANG KURANG, JANGAN ADA YANG LEBIH. TIDAK PERLU ADA ```python.
    """

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )

    # Ambil teks dan parsing hasil list
    raw_output = response.text.strip()
    try:
        labels = eval(raw_output)
        if isinstance(labels, list):
            # Validasi jumlah label
            if len(labels) != len(comments):
                print(f"Jumlah label ({len(labels)}) ≠ jumlah komentar ({len(comments)})")
                labels += ["Error"] * (len(comments) - len(labels))
            return labels
        else:
            raise ValueError("Output bukan list")
    except Exception:
        print("Format output tidak valid, hasil mentah:")
        print(raw_output)
        return ["Error"] * len(comments)

In [40]:
# Contoh penggunaan dengan 5 komentar pertama
comments = df['comment'].head().tolist()
labels = classify_sentiment_batch(comments)

df_preview = pd.DataFrame({
    "comment": comments,
    "sentiment": labels
})

df_preview

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.', 'status': 'RESOURCE_EXHAUSTED'}}

In [ ]:
def process_sentiment_labeling(df, client, batch_size=50, delay=15):
    all_labels = []  # menyimpan semua label hasil model

    # Loop setiap batch komentar
    for i in range(0, len(df), batch_size):
        # Ambil komentar
        batch_comments = df['comment'].iloc[i:i+batch_size].astype(str).tolist()
        print(f"\nMemproses batch {i//batch_size + 1} ({len(batch_comments)} komentar)...")

        try:
            labels = classify_sentiment_batch(batch_comments)
        except Exception as e:
            print(f"Terjadi error pada batch {i//batch_size + 1}: {e}")
            labels = ["Error"] * len(batch_comments)

        # Simpan hasil ke list utama
        all_labels.extend(labels)

        # Jeda antar batch agar tidak limit API request
        if i + batch_size < len(df):
            print(f"Menunggu {delay} detik sebelum lanjut ke batch berikutnya.")
            time.sleep(delay)

    df_result = pd.DataFrame({
        "comments": df["clean_text"],
        "sentiment": all_labels
    })
    
    df_result['sentiment'] = all_labels
    
    return df_result

In [ ]:
df_result = process_sentiment_labeling(df, client, batch_size=50, delay=15)


Memproses batch 1 (100 komentar)...
Jumlah label (70) ≠ jumlah komentar (100)
Menunggu 15 detik sebelum lanjut ke batch berikutnya.


KeyboardInterrupt: 

In [ ]:
# Simpan ke file CSV
df_result[['clean_text','sentiment']].to_csv("dataset_labeled.csv", index=False, encoding="utf-8")

In [11]:
df[['clean_text', 'sentiment']].to_csv("dataset_clean.csv", index=False, encoding='utf-8')